In [ ]:
#Omero

import ezomero
import omero

#Other
import numpy as np
import pandas as pd
import os
import pathlib
import warnings
import glob
import csv
import getpass



#Settings
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# Login to OMERO
OMEROUSER = input(f"Enter username: \t")
OMEROPASS = getpass.getpass(prompt = f"Enter password: \t")


OMEROHOST = ''
OMEROPORT = 
OMEROWEB = ''

In [ ]:
# Connection Check:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)
print(conn.isConnected())

In [ ]:
# Variables
plate_id = 765
filename = r"Z:\Data\@MiN\Jens\OMERO Test Daten\IDR\Metadata\Metadata_for_PopMetadata\bulk_annotations_plate2A.csv"

ns="Experimental Metadata"

In [ ]:
# Get well positions
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True) 
plate = conn.getObject("Plate", plate_id)
list_wellrow, list_wellcolumn, list_wellid, list_wellpos = [], [], [], [] 

print("\nNumber of fields:", plate.getNumberOfFields())
print("\nGrid size:", plate.getGridSize())
print("\nWells in Plate:", plate.getName())
for well in plate.listChildren():
    index = well.countWellSample()
    list_wellid.append(well.getId())
    list_wellpos.append(well.getWellPos())
    list_wellrow.append(well.row)
    list_wellcolumn.append(well.column)

In [ ]:
df_wells = pd.DataFrame()
df_wells["well_id"] = list_wellid
df_wells["Well_Position"] = list_wellpos
df_wells["Well_Row"] = list_wellrow
df_wells["Well_Column"] = list_wellcolumn

In [ ]:
df_annotationdata = pd.read_csv(filename, delimiter=";")
df_annotationdata.head()

In [ ]:
df_annotationdata = df_annotationdata.merge(df_wells, left_on="Well", right_on="Well_Position")
df_annotationdata = df_annotationdata.set_index("well_id")
df_dict = df_annotationdata.to_dict(orient="index")

In [ ]:
#df_dict

In [ ]:
### Annotate Wells only:

conn = ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True) 
for x in df_dict.keys():
    ezomero.post_map_annotation(conn, "Well", x, df_dict[x], ns)

In [ ]:
### Annotate all Images in the wells:

conn = ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True) 
for x in df_dict.keys():
    well = conn.getObject("Well", x)
    index = well.countWellSample()
    for index in range(0, index): # get all image names and ids
        image_id= well.getImage(index).getId()
        ezomero.post_map_annotation(conn, "Image", image_id, df_dict[x], ns)